In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from datetime import datetime as dt
import itertools

%matplotlib inline

In [2]:
df = pd.read_csv("C:/Users/MaurycyOprus/Desktop/UAM - semestr 2/analiza danych sportowych/results.csv")

In [3]:
df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [4]:
df_euro = pd.read_csv("C:/Users/MaurycyOprus/Desktop/UAM - semestr 2/analiza danych sportowych/uefa-euro-2024-UTC.csv")

In [5]:
df_euro.head(5)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Group,Result
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,NaN
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,NaN
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,NaN
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,NaN
4,5,1,16/06/2024 13:00,Volksparkstadion,Play-off Winner A,Netherlands,Group D,NaN


In [6]:
# zakładamy, że z baraży dywizji A wyjdzie Polska, dywizji B - Ukraina, dywizji C - Grecja
df_euro['Home Team'] = df_euro['Home Team'].apply(lambda x: "Poland" if x == "Play-off Winner A" else x)
df_euro['Home Team'] = df_euro['Home Team'].apply(lambda x: "Ukraine" if x == "Play-off Winner B" else x)
df_euro['Home Team'] = df_euro['Home Team'].apply(lambda x: "Greece" if x == "Play-off Winner C" else x)
df_euro['Home Team'] = df_euro['Home Team'].apply(lambda x: "Czech Republic" if x == "Czechia" else x)
df_euro['Home Team'] = df_euro['Home Team'].apply(lambda x: "Turkey" if x == "Türkiye" else x)
df_euro['Away Team'] = df_euro['Away Team'].apply(lambda x: "Poland" if x == "Play-off Winner A" else x)
df_euro['Away Team'] = df_euro['Away Team'].apply(lambda x: "Ukraine" if x == "Play-off Winner B" else x)
df_euro['Away Team'] = df_euro['Away Team'].apply(lambda x: "Greece" if x == "Play-off Winner C" else x)
df_euro['Away Team'] = df_euro['Away Team'].apply(lambda x: "Czech Republic" if x == "Czechia" else x)
df_euro['Away Team'] = df_euro['Away Team'].apply(lambda x: "Turkey" if x == "Türkiye" else x)

In [7]:
df_euro.head(5)

,Match Number,Round Number,Date,Location,Home Team,Away Team,Group,Result
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,NaN
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,NaN
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,NaN
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,NaN
4,5,1,16/06/2024 13:00,Volksparkstadion,Poland,Netherlands,Group D,NaN


In [8]:
euro_teams = pd.concat([df_euro['Home Team']]).unique()

In [9]:
filtered_df = df[df['home_team'].isin(euro_teams) & df['away_team'].isin(euro_teams)]

In [10]:
filtered_df.tail(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
45215,2023-11-18,Germany,Turkey,2,3,Friendly,Berlin,Germany,False
45253,2023-11-20,Ukraine,Italy,0,0,UEFA Euro qualification,Leverkusen,Germany,True
45305,2023-11-21,Austria,Germany,2,0,Friendly,Vienna,Austria,False
45309,2023-11-21,Greece,France,2,2,UEFA Euro qualification,Athens,Greece,False
45314,2023-11-21,Romania,Switzerland,1,0,UEFA Euro qualification,Bucharest,Romania,False


In [11]:
filtered_df['date'] = pd.to_datetime(filtered_df['date'])
filtered_df  = filtered_df[filtered_df['date'].dt.year.between(2015, 2019)]

C:\Users\MaurycyOprus\AppData\Local\Temp\ipykernel_11180\1263563355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['date'] = pd.to_datetime(filtered_df['date'])


In [12]:
filtered_df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
37199,2015-03-27,Spain,Ukraine,1,0,UEFA Euro qualification,Seville,Spain,False
37216,2015-03-28,Netherlands,Turkey,1,1,UEFA Euro qualification,Amsterdam,Netherlands,False
37223,2015-03-29,France,Denmark,2,0,Friendly,Saint-Étienne,France,False
37227,2015-03-29,Hungary,Greece,0,0,UEFA Euro qualification,Budapest,Hungary,False
37231,2015-03-29,Portugal,Serbia,2,1,UEFA Euro qualification,Lisbon,Portugal,False


In [13]:
tours = pd.concat([filtered_df['tournament']]).unique()
tours

array(['UEFA Euro qualification', 'Friendly', 'UEFA Euro',
       'FIFA World Cup qualification', 'FIFA World Cup',
       'UEFA Nations League'], dtype=object)

In [14]:
importance = {'Friendly':1, 'UEFA Euro qualification':3, 'UEFA Nations League':2,
       'FIFA World Cup qualification':4, 'UEFA Euro':5, 'FIFA World Cup':6}

In [15]:
filtered_df['importance'] = filtered_df['tournament'].map(importance)

In [16]:
filtered_df.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,importance
37199,2015-03-27,Spain,Ukraine,1,0,UEFA Euro qualification,Seville,Spain,False,3
37216,2015-03-28,Netherlands,Turkey,1,1,UEFA Euro qualification,Amsterdam,Netherlands,False,3
37223,2015-03-29,France,Denmark,2,0,Friendly,Saint-Étienne,France,False,1
37227,2015-03-29,Hungary,Greece,0,0,UEFA Euro qualification,Budapest,Hungary,False,3
37231,2015-03-29,Portugal,Serbia,2,1,UEFA Euro qualification,Lisbon,Portugal,False,3


In [17]:
def calculate_team_strength(matches_df):
    # Create an empty list to store DataFrames
    dfs = []

    # Iterate through unique teams in both 'home_team' and 'away_team' columns
    unique_teams = pd.concat([matches_df['home_team'], matches_df['away_team']]).unique()

    for team in unique_teams:
        # Filter matches where the team is either the home or away team
        team_matches = matches_df[(matches_df['home_team'] == team) | (matches_df['away_team'] == team)]

        # Calculate strength based on match outcomes
        team_strength = (
            (team_matches['importance'] * 3 * (team_matches['home_team'] == team) +
             team_matches['importance'] * 1 * (team_matches['away_team'] == team) +
             team_matches['importance'] * -1 * ((team_matches['home_team'] == team) & (team_matches['home_score'] < team_matches['away_score']))) +
            (team_matches['importance'] * -1 * ((team_matches['away_team'] == team) & (team_matches['home_score'] > team_matches['away_score'])))
        ).sum()

        # Create a DataFrame with team and total strength
        team_df = pd.DataFrame({'Team': [team], 'Total_Strength': [team_strength]})

        # Append the DataFrame to the list
        dfs.append(team_df)

    # Concatenate all DataFrames in the list
    team_strength_df = pd.concat(dfs, ignore_index=True)

    return team_strength_df

In [18]:
strength_df = calculate_team_strength(filtered_df)

In [19]:
strength_df

,Team,Total_Strength
0,Spain,125
1,Netherlands,71
2,France,190
3,Hungary,90
4,Portugal,167
5,Italy,109
6,Slovakia,98
7,Croatia,159
8,Albania,70
9,Denmark,91


In [20]:
filtered_df_2 = df[df['home_team'].isin(euro_teams) & df['away_team'].isin(euro_teams)]
filtered_df_2['date'] = pd.to_datetime(filtered_df_2['date'])
filtered_df_2  = filtered_df_2[filtered_df_2['date'].dt.year > 2019]
filtered_df_2['importance'] = filtered_df_2['tournament'].map(importance)

C:\Users\MaurycyOprus\AppData\Local\Temp\ipykernel_11180\4162112630.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df_2['date'] = pd.to_datetime(filtered_df_2['date'])


In [21]:
filtered_df_2.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,importance
42067,2020-09-03,Germany,Spain,1,1,UEFA Nations League,Stuttgart,Germany,False,2
42068,2020-09-03,Ukraine,Switzerland,2,1,UEFA Nations League,Lviv,Ukraine,False,2
42070,2020-09-03,Turkey,Hungary,0,1,UEFA Nations League,Sivas,Turkey,False,2
42074,2020-09-03,Slovenia,Greece,0,0,UEFA Nations League,Ljubljana,Slovenia,False,2
42078,2020-09-04,Netherlands,Poland,1,0,UEFA Nations League,Amsterdam,Netherlands,False,2


In [22]:
def update_strengths(matches_df, strengths_df):
    for index, match in matches_df.iterrows():
        home_team = match['home_team']
        away_team = match['away_team']
        home_score = match['home_score']
        away_score = match['away_score']
        importance = match['importance']
        
        # Update the first DataFrame with home_strength and away_strength values
        matches_df.at[index, 'home_strength'] = strengths_df.loc[strengths_df['Team'] == home_team, 'Total_Strength'].values[0]
        matches_df.at[index, 'away_strength'] = strengths_df.loc[strengths_df['Team'] == away_team, 'Total_Strength'].values[0]

        # Update strengths in the second DataFrame based on match outcome
        if home_score > away_score:
            strengths_df.loc[strengths_df['Team'] == home_team, 'Total_Strength'] += 3 * importance
            strengths_df.loc[strengths_df['Team'] == away_team, 'Total_Strength'] += -1 * importance
        elif home_score == away_score:
            strengths_df.loc[strengths_df['Team'] == home_team, 'Total_Strength'] += 1 * importance
            strengths_df.loc[strengths_df['Team'] == away_team, 'Total_Strength'] += 1 * importance
        else:
            strengths_df.loc[strengths_df['Team'] == home_team, 'Total_Strength'] += -1 * importance
            strengths_df.loc[strengths_df['Team'] == away_team, 'Total_Strength'] += 3 * importance

    return matches_df, strengths_df

In [23]:
df_matches, df_strength = update_strengths(filtered_df_2, strength_df)

In [24]:
df_matches.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,importance,home_strength,away_strength
42067,2020-09-03,Germany,Spain,1,1,UEFA Nations League,Stuttgart,Germany,False,2,134.0,125.0
42068,2020-09-03,Ukraine,Switzerland,2,1,UEFA Nations League,Lviv,Ukraine,False,2,104.0,99.0
42070,2020-09-03,Turkey,Hungary,0,1,UEFA Nations League,Sivas,Turkey,False,2,95.0,90.0
42074,2020-09-03,Slovenia,Greece,0,0,UEFA Nations League,Ljubljana,Slovenia,False,2,69.0,56.0
42078,2020-09-04,Netherlands,Poland,1,0,UEFA Nations League,Amsterdam,Netherlands,False,2,71.0,110.0
42082,2020-09-04,Slovakia,Czech Republic,1,3,UEFA Nations League,Bratislava,Slovakia,False,2,98.0,72.0
42086,2020-09-05,Denmark,Belgium,0,2,UEFA Nations League,Copenhagen,Denmark,False,2,91.0,91.0
42087,2020-09-05,Portugal,Croatia,4,1,UEFA Nations League,Porto,Portugal,False,2,167.0,159.0
42094,2020-09-06,Spain,Ukraine,4,0,UEFA Nations League,Madrid,Spain,False,2,127.0,110.0
42095,2020-09-06,Switzerland,Germany,1,1,UEFA Nations League,Basel,Switzerland,False,2,97.0,136.0


In [25]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [26]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

df_matches["team_1"] = le.fit_transform(df_matches["home_team"])
df_matches["team_2"] = le.fit_transform(df_matches["away_team"])

df_matches.head(5)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,importance,home_strength,away_strength,team_1,team_2
42067,2020-09-03,Germany,Spain,1,1,UEFA Nations League,Stuttgart,Germany,False,2,134.0,125.0,8,20
42068,2020-09-03,Ukraine,Switzerland,2,1,UEFA Nations League,Lviv,Ukraine,False,2,104.0,99.0,23,21
42070,2020-09-03,Turkey,Hungary,0,1,UEFA Nations League,Sivas,Turkey,False,2,95.0,90.0,22,10
42074,2020-09-03,Slovenia,Greece,0,0,UEFA Nations League,Ljubljana,Slovenia,False,2,69.0,56.0,19,9
42078,2020-09-04,Netherlands,Poland,1,0,UEFA Nations League,Amsterdam,Netherlands,False,2,71.0,110.0,12,13


In [27]:
def determine_match_result(row):
    if row['home_score'] > row['away_score']:
        return 1  # Home win
    elif row['home_score'] < row['away_score']:
        return 2  # Away win
    else:
        return 0  # Draw

In [28]:
df_matches['result'] = df_matches.apply(determine_match_result, axis=1)

In [94]:
from sklearn.preprocessing import StandardScaler
# Function to calculate strength_diff
def calculate_strength_diff(row):
    return row['home_strength'] - row['away_strength']

df_matches['strength_diff'] = df_matches.apply(calculate_strength_diff, axis=1)

# Standardize the 'strength_diff' values to range -1 to 1
scaler = StandardScaler()
df_matches['strength_diff_standardized'] = scaler.fit_transform(df_matches[['strength_diff']])

In [95]:
df_matches.head(10)

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,importance,home_strength,away_strength,team_1,team_2,result,strength_diff,strength_diff_standardized
42067,2020-09-03,Germany,Spain,1,1,UEFA Nations League,Stuttgart,Germany,False,2,134.0,125.0,8,20,0,9.0,0.077679
42068,2020-09-03,Ukraine,Switzerland,2,1,UEFA Nations League,Lviv,Ukraine,False,2,104.0,99.0,23,21,1,5.0,0.027447
42070,2020-09-03,Turkey,Hungary,0,1,UEFA Nations League,Sivas,Turkey,False,2,95.0,90.0,22,10,2,5.0,0.027447
42074,2020-09-03,Slovenia,Greece,0,0,UEFA Nations League,Ljubljana,Slovenia,False,2,69.0,56.0,19,9,0,13.0,0.127911
42078,2020-09-04,Netherlands,Poland,1,0,UEFA Nations League,Amsterdam,Netherlands,False,2,71.0,110.0,12,13,1,-39.0,-0.525104
42082,2020-09-04,Slovakia,Czech Republic,1,3,UEFA Nations League,Bratislava,Slovakia,False,2,98.0,72.0,18,4,2,26.0,0.291164
42086,2020-09-05,Denmark,Belgium,0,2,UEFA Nations League,Copenhagen,Denmark,False,2,91.0,91.0,5,2,2,0.0,-0.035343
42087,2020-09-05,Portugal,Croatia,4,1,UEFA Nations League,Porto,Portugal,False,2,167.0,159.0,14,3,1,8.0,0.065121
42094,2020-09-06,Spain,Ukraine,4,0,UEFA Nations League,Madrid,Spain,False,2,127.0,110.0,20,23,1,17.0,0.178142
42095,2020-09-06,Switzerland,Germany,1,1,UEFA Nations League,Basel,Switzerland,False,2,97.0,136.0,21,8,0,-39.0,-0.525104


In [105]:
features = ["importance", "home_strength", "away_strength", "strength_diff_standardized", "team_1", "team_2"]
X = df_matches[features]
y = df_matches["result"]

In [106]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=44)

In [107]:
model = RandomForestClassifier(n_estimators=1000, random_state=0)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

In [108]:
y_pred = model.predict(X_test)

In [109]:
accuracy = accuracy_score(y_test, y_pred)
print("Test data accuracy:", accuracy)

Test data accuracy: 0.48148148148148145


In [134]:
df_euro.head(5)

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,NaN
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,NaN
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,NaN
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,NaN
4,5,1,16/06/2024 13:00,Volksparkstadion,Poland,Netherlands,Group D,NaN


In [36]:
def update_strengths_2(matches_df, strengths_df, numbers):
    # Merge matches_df with strengths_df for home team
    matches_df = matches_df.merge(strengths_df[['Team', 'Total_Strength']], left_on='home_team', right_on='Team', how='left')
    matches_df = matches_df.rename(columns={'Total_Strength': 'home_strength'})
    
    # Merge matches_df with strengths_df for away team
    matches_df = matches_df.merge(strengths_df[['Team', 'Total_Strength']], left_on='away_team', right_on='Team', how='left')
    matches_df = matches_df.rename(columns={'Total_Strength': 'away_strength'})
    
    # Merge matches_df with strengths_df for home team
    matches_df = matches_df.merge(numbers[['team', 'team_number']], left_on='home_team', right_on='team', how='left')
    matches_df = matches_df.rename(columns={'team_number': 'team_1'})
    
    # Merge matches_df with strengths_df for away team
    matches_df = matches_df.merge(numbers[['team', 'team_number']], left_on='away_team', right_on='team', how='left')
    matches_df = matches_df.rename(columns={'team_number': 'team_2'})
    
    # Set 'importance' column to 5 for all rows
    matches_df['importance'] = 5
    
    # Drop the redundant 'Team' columns
    matches_df = matches_df.drop(columns=['Team_x', 'Team_y'])
    matches_df = matches_df.drop(columns=['team_x', 'team_y'])
    
    return matches_df

In [135]:
strength_df

,Team,Total_Strength
0,Spain,256
1,Netherlands,150
2,France,353
3,Hungary,160
4,Portugal,255
5,Italy,245
6,Slovakia,111
7,Croatia,233
8,Albania,93
9,Denmark,161


In [136]:
def get_team_numbers(matches_df):
    # Extract unique team names from 'home_team' and 'away_team'
    teams_home = matches_df[['home_team', 'team_1']].rename(columns={'home_team': 'team', 'team_1': 'team_number'})
    teams_away = matches_df[['away_team', 'team_2']].rename(columns={'away_team': 'team', 'team_2': 'team_number'})
    
    # Concatenate and drop duplicates to get unique teams
    unique_teams_df = pd.concat([teams_home, teams_away]).drop_duplicates(subset=['team']).reset_index(drop=True)
    
    return unique_teams_df

In [138]:
df_euro = df_euro.rename(columns={"Home Team": 'home_team'})
df_euro = df_euro.rename(columns={"Away Team": 'away_team'})

In [139]:
df_euro.head(5)

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,NaN
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,NaN
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,NaN
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,NaN
4,5,1,16/06/2024 13:00,Volksparkstadion,Poland,Netherlands,Group D,NaN


In [140]:
numbers_team = get_team_numbers(df_matches)

In [141]:
numbers_team

,team,team_number
0,Germany,8
1,Ukraine,23
2,Turkey,22
3,Slovenia,19
4,Netherlands,12
5,Slovakia,18
6,Denmark,5
7,Portugal,14
8,Spain,20
9,Switzerland,21


In [142]:
euro_2 = update_strengths_2(df_euro, strength_df, numbers_team)

In [143]:
euro_2['strength_diff'] = euro_2.apply(calculate_strength_diff, axis=1)

# Standardize the 'strength_diff' values to range -1 to 1
scaler = StandardScaler()
euro_2['strength_diff_standardized'] = scaler.fit_transform(euro_2[['strength_diff']])

In [144]:
euro_2.head(5)

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result,home_strength,away_strength,team_1,team_2,importance,strength_diff,strength_diff_standardized
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,NaN,208.0,128.0,8.0,16.0,5,80.0,0.735538
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,NaN,160.0,159.0,10.0,21.0,5,1.0,0.065496
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,NaN,256.0,233.0,20.0,3.0,5,23.0,0.252090
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,NaN,245.0,93.0,11.0,0.0,5,152.0,1.346209
4,5,1,16/06/2024 13:00,Volksparkstadion,Poland,Netherlands,Group D,NaN,133.0,150.0,13.0,12.0,5,-17.0,-0.087171


In [145]:
group_matches = euro_2.head(36)

In [146]:
for row in range(group_matches.shape[0]):
    match = group_matches.iloc[[row]].copy()
    prediction = model.predict(match[features])
    euro_2.at[row, 'Result'] = prediction

In [147]:
euro_2

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result,home_strength,away_strength,team_1,team_2,importance,strength_diff,strength_diff_standardized
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,0.0,208.0,128.0,8.0,16.0,5,80.0,0.735538
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,1.0,160.0,159.0,10.0,21.0,5,1.0,0.065496
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,0.0,256.0,233.0,20.0,3.0,5,23.0,0.252090
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,1.0,245.0,93.0,11.0,0.0,5,152.0,1.346209
4,5,1,16/06/2024 13:00,Volksparkstadion,Poland,Netherlands,Group D,2.0,133.0,150.0,13.0,12.0,5,-17.0,-0.087171
5,6,1,16/06/2024 16:00,Arena Stuttgart,Slovenia,Denmark,Group C,2.0,93.0,161.0,19.0,5.0,5,-68.0,-0.519730
6,7,1,16/06/2024 19:00,Arena AufSchalke,Serbia,England,Group C,0.0,80.0,313.0,17.0,6.0,5,-233.0,-1.919184
7,8,1,17/06/2024 13:00,Fußball Arena München,Romania,Ukraine,Group E,1.0,91.0,113.0,15.0,23.0,5,-22.0,-0.129579
8,9,1,17/06/2024 16:00,Frankfurt Stadion,Belgium,Slovakia,Group E,1.0,183.0,111.0,2.0,18.0,5,72.0,0.667686
9,10,1,17/06/2024 19:00,Düsseldorf Arena,Austria,France,Group D,0.0,77.0,353.0,1.0,7.0,5,-276.0,-2.283890


In [148]:
def calculate_group_places(df):
    # Create a dictionary to store group results
    group_results = {}

    # Iterate through each row in the DataFrame
    for index, row in df.iterrows():
        # Check if the match is in a group stage
        if row['Round Number'] != 'Round of 16' and row['Round Number'] != 'Quarter Finals' and row['Round Number'] != 'Semi Finals' and row['Round Number'] != 'Final':
            # Get the group key
            group_key = row['Group']

            # Check if the group key is already in the dictionary
            if group_key not in group_results:
                group_results[group_key] = []

            # Check the result and append the team to the list
            if pd.notna(row['Result']):
                if row['Result'] == 1:
                    group_results[group_key].append((row['home_team'], 3))
                    group_results[group_key].append((row['away_team'], 0))
                elif row['Result'] == 2:
                    group_results[group_key].append((row['home_team'], 0))
                    group_results[group_key].append((row['away_team'], 3))
                else:
                    group_results[group_key].append((row['home_team'], 1))
                    group_results[group_key].append((row['away_team'], 1))

    # Calculate the total points for each team in each group
    group_places = {}
    for group, teams in group_results.items():
        group_places[group] = sorted([(team, sum(points for _, points in teams if team == _)) for team in set(team for team, _ in teams)], key=lambda x: x[1], reverse=True)

    # Print the group results
    for group, teams in group_places.items():
        print(f"\nGroup {group} Places:")
        for place, (team, points) in enumerate(teams, start=1):
            print(f"{place}. {team} - {points} points")

# Call the function with your DataFrame
calculate_group_places(euro_2)


Group Group A Places:
1. Hungary - 7 points
2. Scotland - 4 points
3. Germany - 3 points
4. Switzerland - 1 points

Group Group B Places:
1. Croatia - 5 points
2. Spain - 5 points
3. Italy - 5 points
4. Albania - 0 points

Group Group D Places:
1. France - 7 points
2. Netherlands - 6 points
3. Poland - 3 points
4. Austria - 1 points

Group Group C Places:
1. England - 7 points
2. Denmark - 6 points
3. Slovenia - 3 points
4. Serbia - 1 points

Group Group E Places:
1. Belgium - 9 points
2. Romania - 6 points
3. Slovakia - 3 points
4. Ukraine - 0 points

Group Group F Places:
1. Portugal - 7 points
2. Czech Republic - 6 points
3. Greece - 4 points
4. Turkey - 0 points


In [150]:
euro_2.loc[36:43, 'home_team'] = ['Croatia', 'Hungary', 'Portugal', 'Netherlands', 'Belgium', 'France', 'England', 'Scotland']
euro_2.loc[36:43, 'away_team'] = ['Greece', 'Denmark', 'Italy', 'Romania', 'Germany', 'Czech Republic', 'Poland', 'Spain']
euro_2.loc[36:43, "team_1"] = [3, 10, 14, 12, 2, 7, 6, 16]
euro_2.loc[36:43, "team_2"] = [9, 5, 11, 15, 8, 4, 13, 20]

# Add corresponding home_strength and away_strength values
euro_2.loc[36:43, "home_strength"] = [233.0, 160.0, 255.0, 150.0, 183.0, 353.0, 313.0, 128.0]
euro_2.loc[36:43, "away_strength"] = [55.0, 161.0, 119.0, 91.0, 111.0, 119.0, 133.0, 256.0]

In [152]:
euro_2['strength_diff'] = euro_2.apply(calculate_strength_diff, axis=1)
euro_2['strength_diff_standardized'] = scaler.fit_transform(euro_2[['strength_diff']])

In [189]:
euro_2.head(36)

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result,home_strength,away_strength,team_1,team_2,importance,strength_diff,strength_diff_standardized
0,1,1,14/06/2024 19:00,Fußball Arena München,Germany,Scotland,Group A,0.0,208.0,128.0,8.0,16.0,5,80.0,0.588268
1,2,1,15/06/2024 13:00,Stadion Köln,Hungary,Switzerland,Group A,1.0,160.0,159.0,10.0,21.0,5,1.0,-0.084110
2,3,1,15/06/2024 16:00,Olympiastadion,Spain,Croatia,Group B,0.0,256.0,233.0,20.0,3.0,5,23.0,0.103135
3,4,1,15/06/2024 19:00,BVB Stadion Dortmund,Italy,Albania,Group B,1.0,245.0,93.0,11.0,0.0,5,152.0,1.201069
4,5,1,16/06/2024 13:00,Volksparkstadion,Poland,Netherlands,Group D,2.0,133.0,150.0,13.0,12.0,5,-17.0,-0.237310
5,6,1,16/06/2024 16:00,Arena Stuttgart,Slovenia,Denmark,Group C,2.0,93.0,161.0,19.0,5.0,5,-68.0,-0.671377
6,7,1,16/06/2024 19:00,Arena AufSchalke,Serbia,England,Group C,0.0,80.0,313.0,17.0,6.0,5,-233.0,-2.075712
7,8,1,17/06/2024 13:00,Fußball Arena München,Romania,Ukraine,Group E,1.0,91.0,113.0,15.0,23.0,5,-22.0,-0.279866
8,9,1,17/06/2024 16:00,Frankfurt Stadion,Belgium,Slovakia,Group E,1.0,183.0,111.0,2.0,18.0,5,72.0,0.520180
9,10,1,17/06/2024 19:00,Düsseldorf Arena,Austria,France,Group D,0.0,77.0,353.0,1.0,7.0,5,-276.0,-2.441690


In [157]:
for row in range(36, 44):
    match = euro_2.iloc[[row]].copy()
    prediction = model.predict(match[features])
    euro_2.at[row, 'Result'] = prediction

In [188]:
euro_2.loc[36:43]

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result,home_strength,away_strength,team_1,team_2,importance,strength_diff,strength_diff_standardized
36,37,Round of 16,29/06/2024 16:00,Olympiastadion,Croatia,Greece,NaN,0.0,233.0,55.0,3.0,9.0,5,178.0,1.422358
37,38,Round of 16,29/06/2024 19:00,BVB Stadion Dortmund,Hungary,Denmark,NaN,1.0,160.0,161.0,10.0,5.0,5,-1.0,-0.101132
38,39,Round of 16,30/06/2024 16:00,Arena AufSchalke,Portugal,Italy,NaN,1.0,255.0,119.0,14.0,11.0,5,136.0,1.064891
39,40,Round of 16,30/06/2024 19:00,Stadion Köln,Netherlands,Romania,NaN,1.0,150.0,91.0,12.0,15.0,5,59.0,0.409535
40,41,Round of 16,01/07/2024 16:00,Düsseldorf Arena,Belgium,Germany,NaN,0.0,183.0,111.0,2.0,8.0,5,72.0,0.520180
41,42,Round of 16,01/07/2024 19:00,Frankfurt Stadion,France,Czech Republic,NaN,1.0,353.0,119.0,7.0,4.0,5,234.0,1.898981
42,43,Round of 16,02/07/2024 16:00,Fußball Arena München,England,Poland,NaN,1.0,313.0,133.0,6.0,13.0,5,180.0,1.439380
43,44,Round of 16,02/07/2024 19:00,RB Arena,Scotland,Spain,NaN,2.0,128.0,256.0,16.0,20.0,5,-128.0,-1.182044


In [164]:
euro_2.loc[44:47, 'home_team'] = ['Croatia', 'Portugal', 'Germany', "England"]
euro_2.loc[44:47, 'away_team'] = ['Hungary', 'Netherlands', "France", "Spain"]
euro_2.loc[44:47, "team_1"] = [3, 14, 8, 6]
euro_2.loc[44:47, "team_2"] = [10, 12, 7, 20]

# Add corresponding home_strength and away_strength values
euro_2.loc[44:47, "home_strength"] = [233.0, 255.0, 208.0, 313.0]
euro_2.loc[44:47, "away_strength"] = [160.0, 150.0, 353.0, 256.0]

In [165]:
euro_2['strength_diff'] = euro_2.apply(calculate_strength_diff, axis=1)
euro_2['strength_diff_standardized'] = scaler.fit_transform(euro_2[['strength_diff']])

In [166]:
for row in range(44, 48):
    match = euro_2.iloc[[row]].copy()
    prediction = model.predict(match[features])
    euro_2.at[row, 'Result'] = prediction

In [186]:
euro_2.loc[44:47]

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result,home_strength,away_strength,team_1,team_2,importance,strength_diff,strength_diff_standardized
44,45,Quarter Finals,05/07/2024 16:00,Arena Stuttgart,Croatia,Hungary,NaN,0.0,233.0,160.0,3.0,10.0,5,73.0,0.528691
45,46,Quarter Finals,05/07/2024 19:00,Volksparkstadion,Portugal,Netherlands,NaN,1.0,255.0,150.0,14.0,12.0,5,105.0,0.801046
46,47,Quarter Finals,06/07/2024 16:00,Düsseldorf Arena,Germany,France,NaN,1.0,208.0,353.0,8.0,7.0,5,-145.0,-1.326733
47,48,Quarter Finals,06/07/2024 19:00,Olympiastadion,England,Spain,NaN,0.0,313.0,256.0,6.0,20.0,5,57.0,0.392513


In [168]:
euro_2.loc[48:49, 'home_team'] = ['Croatia', 'Germany']
euro_2.loc[48:49, 'away_team'] = ['Portugal', "Spain"]
euro_2.loc[48:49, "team_1"] = [3, 8]
euro_2.loc[48:49, "team_2"] = [14, 20]

# Add corresponding home_strength and away_strength values
euro_2.loc[48:49, "home_strength"] = [233.0, 208.0]
euro_2.loc[48:49, "away_strength"] = [255.0, 256.0]

In [169]:
euro_2['strength_diff'] = euro_2.apply(calculate_strength_diff, axis=1)
euro_2['strength_diff_standardized'] = scaler.fit_transform(euro_2[['strength_diff']])

In [170]:
for row in range(48, 50):
    match = euro_2.iloc[[row]].copy()
    prediction = model.predict(match[features])
    euro_2.at[row, 'Result'] = prediction

In [185]:
euro_2.loc[48:49]

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result,home_strength,away_strength,team_1,team_2,importance,strength_diff,strength_diff_standardized
48,49,Semi Finals,09/07/2024 19:00,Fußball Arena München,Croatia,Portugal,NaN,0.0,233.0,255.0,3.0,14.0,5,-22.0,-0.279866
49,50,Semi Finals,10/07/2024 19:00,BVB Stadion Dortmund,Germany,Spain,NaN,0.0,208.0,256.0,8.0,20.0,5,-48.0,-0.501155


In [177]:
euro_2.loc[50, 'home_team'] = 'Portugal'
euro_2.loc[50, 'away_team'] = "Germany"
euro_2.loc[50, "team_1"] = [14]
euro_2.loc[50, "team_2"] = [20]

# Add corresponding home_strength and away_strength values
euro_2.loc[50, "home_strength"] = [255.0]
euro_2.loc[50, "away_strength"] = [208.0]

In [178]:
euro_2['strength_diff'] = euro_2.apply(calculate_strength_diff, axis=1)
euro_2['strength_diff_standardized'] = scaler.fit_transform(euro_2[['strength_diff']])

In [183]:
match = euro_2.iloc[[50]].copy()
prediction = model.predict(match[features])
euro_2.at[50, 'Result'] = prediction

In [184]:
euro_2.tail(1)

,Match Number,Round Number,Date,Location,home_team,away_team,Group,Result,home_strength,away_strength,team_1,team_2,importance,strength_diff,strength_diff_standardized
50,51,Final,14/07/2024 19:00,Olympiastadion,Portugal,Germany,NaN,0.0,255.0,208.0,14.0,20.0,5,47.0,0.307402
